# Pip & Import

In [1]:
!pip install pingouin

In [3]:
import numpy as np
import pandas as pd
from statsmodels.distributions.empirical_distribution import ECDF
import pingouin as pg
from sklearn.metrics import roc_auc_score, pairwise_distances
from scipy.stats import iqr
import matplotlib.pyplot as plt
from tqdm import tqdm

# Functions

In [4]:
def get_y_proba(roc, n=100000, prevalence=.5):
  '''Get two arrays, y and proba for a given roc (greater than .5)'''
  n_ones = int(round(n * prevalence))
  n_zeros = n - n_ones
  y = np.array([0] * n_zeros + [1] * n_ones)
  alpha = np.abs(roc - .5) * 2
  proba_zeros = np.linspace(0, 1, n_zeros)
  proba_ones = np.linspace(alpha, 1, n_ones)
  proba = np.concatenate([proba_zeros, proba_ones])
  return y, proba

In [5]:
def distance_quantile(x, q):
  dist = pairwise_distances(np.array(x).reshape(-1,1))
  dist = dist[np.triu_indices(len(dist), k=1)]
  return np.quantile(dist, q)

## Assert

In [6]:
for roc in np.linspace(.5,1,21):
  y, proba = get_y_proba(roc=roc, n=5000, prevalence=.2)
  assert np.abs(roc - roc_auc_score(y, proba)) < .001

In [7]:
assert distance_quantile([1,2,3,4,5,6,7,8,9,10], q=1) == 9

# Distributions for roc 75, 80 and 81

In [8]:
rocs_true = [.75, .80, .81]
rocs_obs = []

for roc_true in tqdm(rocs_true):
  y_universe, proba_universe = get_y_proba(n=100_000, prevalence=.5, roc=roc_true)
  rocs_obs.append([])
  for i in range(5_000):
    index = np.random.choice(range(len(y_universe)), 1_000, replace=True)
    y_sample, proba_sample = y_universe[index], proba_universe[index]
    rocs_obs[-1].append(roc_auc_score(y_sample, proba_sample))

In [9]:
plt.rcParams['legend.title_fontsize'] = 12

fig, ax = plt.subplots()

for model in list(range(len(rocs_obs)))[::-1]:
    ax.hist(rocs_obs[model], bins=np.linspace(.7, .85, 101), alpha=.5, label='{:.2f}'.format(rocs_true[model]))
    
ax.set_title('roc_auc_score (observed on different test sets)', fontsize=12)
ax.set_yticks([])
ax.set_xticks([.7, .725, .75, .775, .8, .825, .85])
ax.set_xticklabels([.7, .725, .75, .775, .8, .825, .85])
ax.xaxis.set_tick_params(labelsize=12)
ax.legend(title='roc_auc_score (true)', fontsize=12, loc='center left', bbox_to_anchor=(1, 0.5))

fig.savefig('three_hist.png', bbox_inches='tight', dpi=200)

# Focus on ROC 80

In [10]:
fig, ax = plt.subplots()
ax.hist(rocs_obs[1], bins=np.linspace(.75, .85, 101))
ax.set_title('roc_auc_score (same model, different test sets)', fontsize=12)
ax.set_yticks([])
ax.xaxis.set_tick_params(labelsize=12)
fig.savefig('single_hist.png', bbox_inches='tight', dpi=200)
print('95th percentile of distance between pairs of ROC:', distance_quantile(rocs_obs[1], .95))

In [11]:
dist = pairwise_distances(np.array(rocs_obs[1]).reshape(-1,1))
dist = dist[np.triu_indices(len(rocs_obs[1]), k=1)]
ecdf = ECDF(dist)

fig, ax = plt.subplots()
ax.hlines(.95, 0, np.quantile(dist,.95), ls="--", color="grey", lw = 2)
ax.vlines(np.quantile(dist,.95), 0, .95, ls="--", color="grey", lw = 2)
ax.plot(ecdf.x, ecdf.y, lw=3)
ax.set_title('ECDF', fontsize=12)
ax.set_xlabel('distance between pairs of roc_auc_score', fontsize=12)
ax.xaxis.set_tick_params(labelsize=12)
ax.yaxis.set_tick_params(labelsize=12)
fig.savefig('ecdf.png', bbox_inches='tight', dpi=200)

# Simulate combinations of ROC, n, prevalence

In [12]:
fig, axs = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(10,8))
fig.subplots_adjust(wspace=.05)

out = pd.DataFrame(columns = ['roc', 'n', 'prevalence', 'dq'])

j = 0

with tqdm() as pbar:
  for roc_true in [.8, .7, .9]:
    y_universe, proba_universe = get_y_proba(n=100_000, prevalence=.5, roc=roc_true)
    for n in [1_000, 5_000, 10_000]:
      for prevalence in [.01, .05, .2]:
        rocs_sample = []
        n_ones = int(round(n * prevalence))
        n_zeros = n - n_ones
        for iter in range(1_000):
          index_zeros = np.random.choice(range(int(len(y_universe)/2)), n_zeros, replace=True)
          index_ones = np.random.choice(range(int(len(y_universe)/2), len(y_universe)), n_ones, replace=True)
          index = np.concatenate([index_zeros, index_ones])
          y_sample = y_universe[index]
          proba_sample = proba_universe[index]
          rocs_sample.append(roc_auc_score(y_sample, proba_sample))
        dq = distance_quantile(rocs_sample, .95)
        out.loc[j, :] = [roc_true, n, prevalence, dq]
        
        if roc_true == .8:
          axs[divmod(j, 3)].hist(rocs_sample, bins=np.linspace(.75,.85,101))
          axs[divmod(j, 3)].legend(title=f'd: {round(dq,3)}',loc='upper left') #, label=
          axs[divmod(j, 3)].set_yticks([])
          axs[divmod(j, 3)].set_title(f'n: {"{:,}".format(n)}, prevalence: {"{:.2f}".format(prevalence)}', fontsize=12)
          axs[divmod(j, 3)].xaxis.set_tick_params(labelsize=12)
          axs[divmod(j, 3)].set_xticks([.75, .775, .8, .825, .85])
          axs[divmod(j, 3)].set_xticklabels([.75, .775, .8, .825, .85])

        j += 1
        pbar.update(1)
    
fig.savefig('nine_hist.png', bbox_inches='tight', dpi=200)
        
out = out.astype(float)

In [17]:
out.columns = ['roc', 'n', 'prevalence', 'd']
out.head()

In [14]:
pcorr = pd.DataFrame()

for var in ['roc', 'n', 'prevalence']:
  y_covar = sorted(set(['roc', 'n', 'prevalence']) - set([var]))
  pcorr_part = pg.partial_corr(data=out, x='roc', y='dq', y_covar=y_covar)
  pcorr_part.index = [var]
  pcorr = pd.concat([pcorr, pcorr_part])
    
pcorr

# Bonus

In [15]:
fig, ax = plt.subplots(figsize=(10,3))
ax.scatter(np.linspace(0,1,20), [1]*20, color='blue')
ax.scatter(np.linspace(.6,1,20), [0]*20, color='red')
ax.text(.8,0.3,'positives (evenly spaced between α and 1)', ha='center', va='center', fontsize=14)
ax.text(.5,1.3,'negatives (evenly spaced between 0 and 1)', ha='center', va='center', fontsize=14)
ax.set_ylim(-1,2)
ax.set_xlim(-.05,1.15)
ax.set_yticks([])
ax.xaxis.set_tick_params(labelsize=14)

fig.savefig('y_proba_from_roc.png', bbox_inches='tight', dpi=200)